In [2]:
!nvidia-smi

Sat Dec 27 18:46:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   35C    P0             55W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Load the GTZAN dataset for genre classification

In [3]:
%pip install torch torchcodec transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.8 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

gtzan = load_dataset("ParitKansal/marsyas-gtzan")
gtzan

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/630 [00:00<?, ?B/s]

data/train-00000-of-00003.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

data/train-00001-of-00003.parquet:   0%|          | 0.00/429M [00:00<?, ?B/s]

data/train-00002-of-00003.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/999 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

Create the validation set ourselves, since this dataset doesn't have a predefined validation set

In [5]:
gtzan = gtzan["train"].train_test_split(shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

Inspect one of the audio files

In [6]:
gtzan["train"][0]

{'file': 'C:\\Users\\xelpmoc\\.cache\\huggingface\\datasets\\downloads\\extracted\\34ed3ff362ee8d2b8e650be17a5643e8a91e2f825b67f3c2dd42604129683e77\\genres\\reggae\\reggae.00029.wav',
 'audio': <datasets.features._torchcodec.AudioDecoder at 0x7dd4b34782f0>,
 'genre': 8}

Map genre ids to human readable names

In [7]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'reggae'

Listen to a few examples using gradio

In [8]:
import gradio as gr


def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

# demo.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/processing_utils.py:688: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/usr/local/lib/python3.12/dist-packages/gradio/processing_utils.py:688: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/usr/local/lib/python3.12/dist-packages/gradio/processing_utils.py:688: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/usr/local/lib/python3.12/dist-packages/gradio/processing_utils.py:688: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Start playing with the feature extractor

In [9]:
from transformers import AutoFeatureExtractor

model_id = "m-a-p/MERT-v1-330M"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Obtain the models native sampling rate from the feature extractor

In [10]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

24000

Resample the dataset to the model's native sampling rate

In [11]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

Confirm the first sample is set to the models native sampling rate

In [12]:
gtzan["train"][0]["audio"].get_all_samples()

AudioSamples:
  data (shape): torch.Size([1, 720320])
  pts_seconds: 0.0
  duration_seconds: 30.013333333333332
  sample_rate: 24000

Compute the mean and variance for the first audio sample. The goal is to achieve zero mean unit variance

In [13]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: -1.95e-05, Variance: 0.0126


Apply the feature extractor

In [14]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -3.77e-09, Variance: 1.0


Ensure samples do not exceed 30 seconds of run time

In [15]:
max_duration = 5.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

Apply data sanitization to the dataset

In [16]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

Rename the genre column to labels for the `Trainer` class

In [17]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

Convert genre labels to human readable labels

In [18]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

Load the pretrained model

In [19]:
from transformers import HubertForSequenceClassification, AutoConfig

model_id = "m-a-p/MERT-v1-330M"

# 1. Load MERT's config
config = AutoConfig.from_pretrained(model_id, num_labels=10, trust_remote_code=True)

# 2. Trick Hugging Face into thinking it's a standard HuBERT model
# This enables the "PreTrainedModel" magic (saving configs, auto-generating READMEs)
model = HubertForSequenceClassification.from_pretrained(
    model_id,
    config=config,
    ignore_mismatched_sizes=True,
    trust_remote_code=True
)

config.json: 0.00B [00:00, ?B/s]

configuration_MERT.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/m-a-p/MERT-v1-330M:
- configuration_MERT.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at m-a-p/MERT-v1-330M and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Log into the hub to sync checkpoints during training, and for weights and biases like analysis

In [20]:
from huggingface_hub import notebook_login

notebook_login()

Define training arguments

In [22]:
from transformers import TrainingArguments

model_name = "m-a-p/MERT-v1-330M"
batch_size = 16
gradient_accumulation_steps = 1
num_train_epochs = 15

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # added cosine annealing
    lr_scheduler_type="cosine",
    fp16=True,
    push_to_hub=True,
)

Use accuracy as the metric and evaluate with the huggingface evaluate library

In [23]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00


In [24]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

Train the model!

In [25]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-2059408770.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose "Don't visualize my results"


wandb: Detected [huggingface_hub.inference, mcp] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Epoch,Training Loss,Validation Loss,Accuracy
1,2.125000,2.096660,0.460000
2,1.186200,1.073892,0.730000
3,0.705500,0.774362,0.810000
4,0.445000,0.528296,0.860000
5,0.278800,0.484754,0.890000
6,0.094800,0.519220,0.890000
7,0.070900,0.463801,0.900000
8,0.081800,0.434480,0.910000
9,0.083500,0.463821,0.920000
10,0.006300,0.417455,0.940000


TrainOutput(global_step=855, training_loss=0.3789993910630893, metrics={'train_runtime': 1884.5642, 'train_samples_per_second': 7.156, 'train_steps_per_second': 0.454, 'total_flos': 3.065135815850401e+18, 'train_loss': 0.3789993910630893, 'epoch': 15.0})

Push to hub

In [26]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...d-gtzan/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...61369.d7153b8b94f2.1124.0: 100%|##########| 48.0kB / 48.0kB            

  ...d-gtzan/model.safetensors:   3%|3         | 41.9MB / 1.26GB            

CommitInfo(commit_url='https://huggingface.co/dzur658/MERT-v1-330M-finetuned-gtzan/commit/160034a919fc6a85f8ba075562f762744fdf53e8', commit_message='End of training', commit_description='', oid='160034a919fc6a85f8ba075562f762744fdf53e8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dzur658/MERT-v1-330M-finetuned-gtzan', endpoint='https://huggingface.co', repo_type='model', repo_id='dzur658/MERT-v1-330M-finetuned-gtzan'), pr_revision=None, pr_num=None)